# Part 1

In [5]:
import numpy as np

class Convolution:
    def __init__(self, input_dim, conv_dim, W=None):
        self.input_dim = input_dim
        self.conv_dim = conv_dim
        self.W = np.random.uniform(0.4, 0.6, conv_dim) if W is None else W
        self.input_values = None
        self.output_values = None
        self.W_pd = None

    def convolve(self, T, W, add_padding=False):
        T = np.expand_dims(T, axis=0) if T.ndim == 2 else T
        W = np.expand_dims(W, axis=0) if W.ndim == 2 else W
        output_shape = (1, (T.shape[1] + W.shape[1] - 1), (T.shape[2] + W.shape[2] - 1)) if add_padding else (1, (T.shape[1] - W.shape[1] + 1), (T.shape[2] - W.shape[2] + 1))
        T = np.pad(T, pad_width=[(0, 0), (W.shape[1] - 1, W.shape[1] - 1), (W.shape[2] - 1, W.shape[2] - 1)]) if add_padding else T

        convolution = np.zeros(output_shape)
        for row in range(output_shape[1]):
            for col in range(output_shape[2]):
                convolution[0, row, col] = np.sum(T[:, row: row + W.shape[1], col: col + W.shape[2]] * W)
        return convolution

    def forward(self, input, labels=None):
        self.input_values = self.change_dims(input, self.input_dim)
        self.output_values = self.convolve(self.input_values, self.W, add_padding=True)
        return self.change_dims(self.output_values, self.output_dim)

    def backward(self, input_pd):
        self.W_pd = np.concatenate([self.convolve(self.input_values[i], self.change_dims(input_pd, self.input_dim), add_padding=False) for i in range(self.n_input[0])], axis=0)
        return self.change_dims(np.concatenate([self.convolve(self.change_dims(input_pd, self.input_dim), self.W[i, ::-1, ::-1], add_padding=True) for i in range(self.n_input[0])], axis=0), self.output_dim)

    def optimize_weights(self, gd):
        self.W = gd.optimize([self.W], [self.W_pd])[0]

    def change_dims(self, X, dim):
        pass

# Part 2

In [6]:
def get_padding_and_filter_size(input_size, output_size):
    padding = (input_size - output_size) // 2
    return padding, input_size - 2 * padding

In [7]:
input_size = 32
output_size = 28

padding, filter_size = get_padding_and_filter_size(input_size, output_size)
print(f"Padding: {padding}, filter size: {filter_size}")

Padding: 2, filter size: 28
